In [ ]:
import os
import sys
import pandas as pd
import numpy as np
from datetime import date
import matplotlib.pyplot as plt
import math

In [ ]:
os.getcwd()

In [ ]:
os.chdir('C:/Users/FGetachew/jupyter_workspace/bwt')

In [ ]:
df = pd.read_csv('view_tpgs_farmer_bodyweight_combine.csv')

In [ ]:
df

In [ ]:
type(df)

*Exploring the data*

In [ ]:
#see largest maximum number of columns with head
pd.set_option('display.max_columns', 500)

In [ ]:
df.columns

In [ ]:
df.head(50)

In [ ]:
df.tail()

In [ ]:
df.sample(5) # draw a random number of 5 rows from the dataset

In [ ]:
df.index

In [ ]:
#more information about the data type and memory it used e.g. 2.6GB
df.info(verbose = False)
#df.info()

In [ ]:
df.describe()

In [ ]:
df[['adminlevel1_name']].describe()

In [ ]:
df.shape
#len(df)

In [ ]:
df[['adminlevel1_name', 'adminlevel2_name', 'adminlevel3_name','adminlevel4_name']]

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
#subsetting a dataframe
df[df.columns[5:10]]

In [ ]:
#subsetting a dataframe
#df[df.columns[5:10]]
df[df.columns[-5:]] #last five columns

In [ ]:
df.select_dtypes?

# Data wrangling

In [ ]:
df = df[df['country_name'] == 'Ethiopia']
# df = df[df['farmer_introbirdstrainname'] == 'SASSO']

df.shape

In [ ]:
df = df[[
    'farmer_introbirdstrainname',
    'form_datecollection',
    'farmer_introbirdstrain_hatchdate',
    'adminlevel1_name',
    'adminlevel2_name', 
    'adminlevel3_name', 
    'adminlevel4_name',
    'farmer_fullname',
    'introbirds_ageindays',
    'introbirds_ageinweeks',
    'introduced_all_count',	
    'introduced_all_weight', 
    'introduced_male_count', 
    'introduced_male_weight', 
    'introduced_female_count',	
    'introduced_female_weight'
]]

In [ ]:
display(df[df.duplicated()].shape)
df.drop_duplicates(inplace=True)
display(df[df.duplicated()].shape)
display(df.shape)

In [ ]:
df = df.drop(df[df['farmer_fullname'].isna()].index)

In [ ]:
df.insert(loc=0, column='farmer_id', value=df.set_index([
    'adminlevel1_name',
    'adminlevel2_name', 
    'adminlevel3_name', 
    'adminlevel4_name',
    'farmer_fullname',
]).index.factorize()[0]+1)

df['farmer_id'].unique().shape

In [ ]:
df['c_age_in_days'] = (df['form_datecollection'].astype('datetime64[ns]') - df['farmer_introbirdstrain_hatchdate'].astype('datetime64[ns]')).dt.days
df[df['c_age_in_days'] != df['introbirds_ageindays']].shape

In [ ]:
both_empty = df[
    (df['introduced_male_weight'] == 0) &
    (df['introduced_female_weight'] == 0)
]

df = df.drop(both_empty.index, axis=0)

In [ ]:
# Drop Empty in introbirds_ageindays
df = df.drop(df[df['introbirds_ageindays'].isnull()].index, axis=0)
df = df.drop(df[df['introbirds_ageindays'] < 0].index, axis=0)

df.shape

In [ ]:
df['intor_male_avg'] = df['introduced_male_weight'] / df['introduced_male_count']
df['intor_female_avg'] = df['introduced_female_weight'] / df['introduced_female_count']

In [ ]:
df.groupby('introbirds_ageinweeks').size().plot(kind="bar",figsize=(27, 10))

In [ ]:
df[['introbirds_ageinweeks', 'introduced_female_weight']]['introbirds_ageinweeks'].max()

In [ ]:
display(df['intor_male_avg'].describe())
display(df['intor_female_avg'].describe())

In [ ]:
# Remove average body weight not in between 40 & 4000

min_body_weight = 40
max_body_weight = 4000

m_err = df[
    (df['intor_male_avg'] < min_body_weight) |
    (df['intor_male_avg'] > max_body_weight)
]

display(m_err.shape)
df = df.drop(m_err.index, axis=0)


f_err = df[
    (df['intor_female_avg'] < min_body_weight) |
    (df['intor_female_avg'] > max_body_weight)
]

display(f_err.shape)
df = df.drop(f_err.index, axis=0)

In [ ]:
display(df['intor_male_avg'].describe())
display(df['intor_female_avg'].describe())

In [ ]:
# Duplicate rows
df_duplicates = df[df.duplicated(['farmer_id', 'introbirds_ageindays'], keep=False)].sort_values('farmer_id')

display(df_duplicates.shape)
df_duplicates.to_csv('./duplicates.csv')

In [ ]:
# Take rows with max chicken count

df['total_chicken_count'] = df['introduced_male_count'] + df['introduced_female_count']
df_groups = df.groupby(['farmer_id', 'farmer_fullname', 'introbirds_ageindays'])
df_groups.size().plot(kind='hist')

In [ ]:
df_groups_max = df_groups.max('total_chicken_count')
# df_clean = df_groups_max.reset_index().drop('total_chicken_count', axis=1)
df_clean = df_groups_max.reset_index()

In [ ]:
# df_clean[df_clean.duplicated(['farmer_id', 'introbirds_ageinweeks'])]

In [ ]:
# Max week/range => 88 / 3 week

df_dist = df_clean

week_diff =  math.floor(
    (df_dist['introbirds_ageinweeks'].max() - df_dist['introbirds_ageinweeks'].min()) / 6
)

# ranges = (range(
#     int(df['introbirds_ageinweeks'].min()), 
#     int(df['introbirds_ageinweeks'].max()) + week_diff, 
#     week_diff
# ))
ranges = [8, 12, 16, 20]


# Week diffrence
display(week_diff)
display(ranges)
df_dist.groupby(pd.cut(df_dist['introbirds_ageinweeks'], ranges)).size().plot(kind="bar")

In [ ]:
c = df.groupby(['farmer_id', 'total_chicken_count'])
c.size().plot(kind='hist')

In [ ]:
# Number of chicken in a given week
# df_dist.groupby(
#     pd.cut(df_dist['introbirds_ageinweeks'], ranges), 
#     as_index=False)['total_chicken_count'].agg(['size', 'sum']).plot(kind='bar',x='introbirds_ageinweeks', y='sum')
df_dist.groupby(
    pd.cut(df_dist['introbirds_ageinweeks'], ranges), 
    as_index=False)['total_chicken_count'].agg(['size', 'sum']).plot(kind='bar',x='introbirds_ageinweeks', y='sum')

In [ ]:
# Male count
df_dist.groupby(
    pd.cut(df_dist['introbirds_ageinweeks'], ranges), 
    as_index=False)['introduced_male_count'].agg(['size', 'sum']).plot(kind='bar',x='introbirds_ageinweeks', y='sum')

# Male count
df_dist.groupby(
    pd.cut(df_dist['introbirds_ageinweeks'], ranges), 
    as_index=False)['introduced_female_count'].agg(['size', 'sum']).plot(kind='bar',x='introbirds_ageinweeks', y='sum')

In [ ]:
df_dist.groupby('introbirds_ageinweeks', 
    as_index=False)['introduced_female_count'].agg(['size', 'sum']).plot(kind='bar',x='introbirds_ageinweeks', y='sum')


In [ ]:
# Average body weight
df_dist.groupby(
    pd.cut(df_dist['introbirds_ageinweeks'], ranges), 
    as_index=False)['intor_male_avg'].agg(['size', 'mean']).plot(kind='line',x='introbirds_ageinweeks', y='mean')

df_dist.groupby(
    pd.cut(df_dist['introbirds_ageinweeks'], ranges), 
    as_index=False)['intor_female_avg'].agg(['size', 'mean']).plot(kind='line',x='introbirds_ageinweeks', y='mean')

In [ ]:
pd.pivot(
    df_clean, 
    index=['farmer_id'], 
    columns=["introbirds_ageindays"], 
    values=['introduced_male_count', 'introduced_male_weight']
    )

In [ ]:
# df_duplicates = df[df.duplicated([
#     'country_name',
#     'adminlevel1_name',
#     'adminlevel2_name', 
#     'adminlevel3_name', 
#     'adminlevel4_name',
#     'farmer_fullname', 
#     'introbirds_ageinweeks'])]

# df_duplicates

# df[df.duplicated([
#     'farmer_fullname', 
#     'introbirds_ageinweeks'])][['farmer_fullname', 'introbirds_ageinweeks']].head(40)


In [ ]:
# df_duplicates[df_duplicates['introduced_male_count'] == 0]

In [ ]:
# df[df.duplicated([
#     'adminlevel1_name',
#     'adminlevel2_name', 
#     'adminlevel3_name', 
#     'adminlevel4_name', 
#     'farmer_fullname', 
#     'introbirds_ageinweeks',
#     'farmer_introbirdstrain'])]['farmer_fullname'].unique().shape

In [ ]:
# # Total Duplicates
# display(df[df.duplicated()].shape)

# df.drop_duplicates(inplace=True)

# display(df[df.duplicated()].shape)

In [ ]:
# display(df[df.duplicated(['farmer_code', 'introbirds_ageinweeks'])].shape)
# display(df[df.duplicated(['farmer_code', 'introbirds_ageinweeks', 'introduced_male_weight'])].shape)
# display(df[df.duplicated(['farmer_code', 'introbirds_ageinweeks', 'introduced_male_count'])].shape)

In [ ]:
# df[df.duplicated(['farmer_code', 'introbirds_ageinweeks'])]['farmer_code'].unique()



In [ ]:
# df_weight = pd.pivot(df, index=['farmer_code'], columns="introbirds_ageinweeks", values='introduced_male_weight')